# Downloading GUNW products using ariaDownload.py

**Author**: Brett A. Buzzanga, David Bekaert - Jet Propulsion Laboratory

This notebook demonstrates how to use the productAPI.py command line tool to download Sentinel 1 ARIA Geocoded UNWrapped interferogram (**GUNW**) products.  A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the [ARIA website](https://aria.jpl.nasa.gov/node/97).

The **`ariaDownload.py`** program wraps around the NASA's ASF DAAC API and [Bulk Download Service](https://bulk-download.asf.alaska.edu/help). The ASF Bulk Download Service handles most of the heavy lifting of the data-download and will conveniently skip previously downloaded files, and re-download partially downloaded files.  
In this notebook, we will demonstrate **`ariaDownload.py`** functionality along track 4, which intersects the U.S. East Coast in southeastern Virginia.


<div class="alert alert-warning">
<b>Potential download failure:</b> 
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add "GRFN Door (PROD)" to their URS approved applications

<b>Login Credentials:</b>
Save your user-name and password to a `.netrc` file in your `$HOME` directory, or pass the combination explicitly using `ariaDownload.py --user <user> --pass <pass>`.


To create a .netrc file, pass your earthdata credentials by running the cell below
</div>

In [ ]:
import os

# create .netrc if it does not exist    
if not os.path.exists('~/.netrc'):
    print('NEEDED To Download ARIA GUNWs: \n Link to create account : https://urs.earthdata.nasa.gov/')
    earthdata_user = input('Please type your Earthdata username:')
    earthdata_user = str(earthdata_user)
    earthdata_password = input('Please type your Earthdata password:')
    earthdata_password = str(earthdata_password)
    os.system('echo machine urs.earthdata.nasa.gov login "{usern}" password "{passwd}" > ~/.netrc; chmod 600 ~/.netrc'.format( \
              usern = earthdata_user, passwd = earthdata_password))

## Overview of the ariaDownload.py program

Running **`ariaDownload.py`** with no options, or with **`-h`**, will show the parameters options as well as some examples. At minimum, users need to specify a spatial constraint either as a track number or bounding box (can be a shapefile).

Let us explore what some of the other options are:

In [1]:
!ariaDownload.py

ARIA-tools Version: 1.1.0
usage: ariaDownload.py [-h] [-o {Download,Count,Url}] [-t TRACK] [-b BBOX]
                       [-w WD] [-s START] [-e END] [-u USER] [-p PASSW]
                       [-l DAYSLT] [-m DAYSGT] [-nt NUM_THREADS] [-i IFG]
                       [-d FLIGHTDIR] [--version VERSION] [-v]

Command line interface to download GUNW products from the ASF DAAC. GUNW products are hosted at the NASA ASF DAAC.

optional arguments:
  -h, --help            show this help message and exit
  -o {Download,Count,Url}, --output {Download,Count,Url}
                        Output type. Default="Download". Use "Url" for
                        ingestion to aria*.py
  -t TRACK, --track TRACK
                        track to download; single number (including leading
                        zeros) or comma separated
  -b BBOX, --bbox BBOX  Lat/Lon Bounding SNWE, or GDAL-readable file
                        containing POLYGON geometry.
  -w WD, --workdir WD   Specify directory to depo

## 1. Example of ariaDownload output options

The user has the option to **download** or **count** the number of products corresponding to return of their search criteria. Below we provide example for each of these over Track 4.

### Count the number of products

To get a count of the number of products, without downloading data, provide the **`--output`** option with the **`count`** argument. To get information on the exact product filenames also include the verbose option **`-v`**.

In [2]:
!ariaDownload.py --track 4 --output count

ARIA-tools Version: 1.1.0

Found -- 3907 -- products


### Generate list of virtual products from ASF S3 bucket (BETA)

To generate a textfile containing a list of product URLs from the ASF S3 bucket, without downloading data, provide the **`--output`** option with the **`url`** argument. To get information on the exact product filenames also include the verbose option **`-v`**. Extracting layers virtually by leveraging this list of URLs is currently only supported by systems with the following packages: Linux kernel >=4.3 and libnetcdf >=4.5 

In [3]:
!ariaDownload.py --track 4 --output url

ARIA-tools Version: 1.1.0
Wrote -- 3907 -- product urls to: /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaDownload/products/track4_1.txt


We can now have a look at the generated textfile, which contains the URLs of all standard products over the specified track. As there are a lot of products we will only have a look at the first 10.

In [4]:
!head -n 10 products/track4_0.txt

https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210805_20210712-230851-45103N_43235N-PP-38f8-v2_0_4.nc
https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210805_20210712-230827-43613N_41745N-PP-e5b8-v2_0_4.nc
https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210805_20210712-230801-42125N_40255N-PP-80bd-v2_0_4.nc
https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210805_20210712-230737-40635N_38763N-PP-3e83-v2_0_4.nc
https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210805_20210712-230712-39145N_37270N-PP-1db5-v2_0_4.nc
https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210805_20210712-230647-37654N_35777N-PP-8a73-v2_0_4.nc
https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210805_20210712-230622-36161N_34118N-PP-39da-v2_0_4.nc
https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20210712_20210606-230850-45103N_43235N-PP-0ce6-v2_0_4.nc
https://grfn.asf.alaska.edu/door/downloa

### Download products

To download the products, simply omit the **`--output`** option or specify it with the **`download`** argument. Examples for downloading all data on track 4 include:

`ariaDownload.py -t 004`

`ariaDownload.py -t 004 -o download`

If user/pass are not explicitly passed, the wrapper will look for a `.netrc` in the `$HOME` directory. Alternatively, you can pass them directly as `ariaDownload.py --user <USER> --pass <PASS>`. Otherwise, you will be prompted to enter them.

By default, the products will be downloaded into the *./products* folder. You can specify a different location using the **`-w`** option, such as:

`ariaDownload.py -t 004 -w /insarHome`

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add "GRFN Door (PROD)" to their URS approved applications
</div>

## 2. Spatial-temporal subsetting

The **`ariaDownload.py`** program has a number of options availble for subsetting the search of products in the spatial and temporal domain, including **bounding box**, **start/end date**, **aquisition geometry**, **temporal baseline**, or **interferogram** options. Below, using the count option for brevity, we demonstrate each of them.

### Subsetting by Bounding Box

Bounding box can either be specified as **SNWE** coordinates (a string with quotation) or by providing a **shapefile** (GeoJSON or ESRI shapefiles) to the **`--bbox`** option. 

An example based on subsetting using **SNWE** coordinates and its location are shown in **Fig. 2**:

In [5]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count

ARIA-tools Version: 1.1.0

Found -- 551 -- products


<img src="./support_docs/crop_hr.png" alt="crop" width="700">

<center><b>Fig. 2</b> Image of products available along within bounding box 36.75S, 37.225N, -76.655W, -75.928E</center>

An example based on subsetting using a shape file:

In [6]:
!ariaDownload.py --track 4 --bbox './support_docs/HR_North.shp' -o count

ARIA-tools Version: 1.1.0

Found -- 1092 -- products


<img src="./support_docs/crop_shp.png" alt="crop" width="700">

<center><b>Fig. 3</b> Image of products available along within shapefile (blue shading). </center>

### Subsetting by start/end date

Subsetting in time can be done by specifying a range within which products need to fall. This is controlled using the **`--start YYYYMMDD` ** and **`--end YYYYMMDD`** options. By default the complete observational record is considered.

In [7]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --start 20190101 

ARIA-tools Version: 1.1.0

Found -- 195 -- products


In [8]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count -s 20190101 -e 20190401 --verbose

ARIA-tools Version: 1.1.0

Found -- 12 -- products
S1-GUNW-A-R-004-tops-20190325_20190217-230628-37654N_35777N-PP-0b9f-v2_0_4
S1-GUNW-A-R-004-tops-20190325_20190301-230628-37654N_35777N-PP-13e7-v2_0_2
S1-GUNW-A-R-004-tops-20190325_20190313-230628-37654N_35777N-PP-7e46-v2_0_2
S1-GUNW-A-R-004-tops-20190313_20190124-230627-37655N_35778N-PP-06c9-v2_0_4
S1-GUNW-A-R-004-tops-20190313_20190217-230627-37655N_35778N-PP-7b05-v2_0_2
S1-GUNW-A-R-004-tops-20190313_20190301-230627-37655N_35778N-PP-e293-v2_0_2
S1-GUNW-A-R-004-tops-20190301_20190112-230628-37655N_35778N-PP-a4ed-v2_0_4
S1-GUNW-A-R-004-tops-20190301_20190124-230628-37655N_35778N-PP-d26d-v2_0_2
S1-GUNW-A-R-004-tops-20190301_20190217-230628-37655N_35778N-PP-8e7c-v2_0_4
S1-GUNW-A-R-004-tops-20190217_20190112-230628-37654N_35778N-PP-0810-v2_0_2
S1-GUNW-A-R-004-tops-20190217_20190124-230628-37654N_35778N-PP-7fcc-v2_0_2
S1-GUNW-A-R-004-tops-20190124_20190112-230628-37654N_35778N-PP-2cb6-v2_0_2


Note the use of the **`--verbose`** option to show the products found.

### Subsetting based on product temporal baseline

The **`--daysless`** and **`--daysmore`** options allow us to subset based on the temporal baseline, i.e. the time length (in days) between the reference and secondary SAR acquisitions of the interferogram. Note that `--daysless 12` and `--daysmore 12` specifies only pairs with temporal baselines smaller and larger than 12 (so not exactly separated by 12) are downloaded, respectively.

To select only interferograms products with temporal baselines shorter than 13 days:

In [9]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysless 13 -v

ARIA-tools Version: 1.1.0

Found -- 118 -- products
S1-GUNW-A-R-004-tops-20210618_20210606-230644-37654N_35777N-PP-31c8-v2_0_4
S1-GUNW-A-R-004-tops-20201220_20201208-230642-37654N_35778N-PP-e453-v2_0_4
S1-GUNW-A-R-004-tops-20201208_20201126-230643-37655N_35778N-PP-8130-v2_0_4
S1-GUNW-A-R-004-tops-20201126_20201114-230643-37655N_35778N-PP-6fb0-v2_0_4
S1-GUNW-A-R-004-tops-20201114_20201102-230643-37654N_35778N-PP-c88d-v2_0_4
S1-GUNW-A-R-004-tops-20201102_20201021-230644-37654N_35778N-PP-6e54-v2_0_4
S1-GUNW-A-R-004-tops-20201021_20201009-230644-37655N_35778N-PP-acd0-v2_0_4
S1-GUNW-A-R-004-tops-20201009_20200927-230644-37655N_35778N-PP-e1bd-v2_0_4
S1-GUNW-A-R-004-tops-20200927_20200915-230643-37654N_35778N-PP-c8fc-v2_0_4
S1-GUNW-A-R-004-tops-20200915_20200903-230643-37655N_35779N-PP-91bd-v2_0_4
S1-GUNW-A-R-004-tops-20200903_20200822-230643-37655N_35779N-PP-b489-v2_0_4
S1-GUNW-A-R-004-tops-20200822_20200810-230642-37655N_35778N-PP-210b-v2_0_4
S1-GUNW-A-R-004-tops-20200810_20200729-230641-37

Select only pairs with temporal baselines, greater than a year:

In [10]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysmore 364

ARIA-tools Version: 1.1.0

Found -- 177 -- products


Select only pairs with moderate temporal baselines between two and six months:

In [11]:
!ariaDownload.py --track 4  --bbox "36.75 37.225 -76.655 -75.928"  -o count -m 59 -l 181 -v 

ARIA-tools Version: 1.1.0

Found -- 10 -- products
S1-GUNW-A-R-004-tops-20190605_20190406-230631-37654N_35777N-PP-6072-v2_0_4
S1-GUNW-A-R-004-tops-20180306_20180105-230621-37654N_35778N-PP-ddb3-v2_0_4
S1-GUNW-A-R-004-tops-20180222_20171224-230621-37655N_35778N-PP-dd1e-v2_0_4
S1-GUNW-A-R-004-tops-20161030_20160702-230618-37654N_35617N-PP-22b0-v2_0_4
S1-GUNW-A-R-004-tops-20161018_20160702-230618-37656N_35619N-PP-199b-v2_0_2
S1-GUNW-A-R-004-tops-20161006_20160409-230618-37655N_35618N-PP-8288-v2_0_2
S1-GUNW-A-R-004-tops-20161006_20160702-230618-37655N_35618N-PP-6382-v2_0_2
S1-GUNW-A-R-004-tops-20160702_20160409-230614-37655N_35618N-PP-f253-v2_0_2
S1-GUNW-A-R-004-tops-20150825_20150310-230613-37822N_35782N-PP-40a2-v2_0_1
S1-GUNW-A-R-004-tops-20150825_20150310-230613-37822N_35782N-PP-43bb-v2_0_4


### Subsetting based on interferogram pair

To find a specific interferogram combination one can use the **`--ifg YYYYMMDD_YYYYMMDD`** option.

In [12]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"  -o count --ifg "20161018_20160702" -v 

ARIA-tools Version: 1.1.0

Found -- 1 -- products
S1-GUNW-A-R-004-tops-20161018_20160702-230618-37656N_35619N-PP-199b-v2_0_2


### Testing your URS credentials set-up

We will now verify if all is set for you to download products, per instructions at the top of the notebook.

In [18]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"  --ifg "20161018_20160702" -v 

ARIA-tools Version: 1.1.0
/Users/buzzanga/Miniconda3/envs/ARIA/lib/python3.9/site-packages/asf_search/download/download.py:62: UserWarning: File already exists, skipping download: /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaDownload/products/S1-GUNW-A-R-004-tops-20161018_20160702-230618-37656N_35619N-PP-199b-v2_0_2.nc
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
Wrote -- 1 -- products to: /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaDownload/products
S1-GUNW-A-R-004-tops-20161018_20160702-230618-37656N_35619N-PP-199b-v2_0_2
